# Course 9 Week 4 Capstone Project - Location Value Calculation for Pharmacies in the City of Zurich


## This Notebook is for Marketing Leaders and Investors of the Pharmacy Business.

### Introduction

Success for pharmacies depends much on their location. The location is the most important factor for the creation of revenue. 
Here I try to quantify the location value of pharmacies at the City of Zurich.
The location value for is among other factors determined by:
- The locations of competitors like other Pharmacies and stores that also sell Parfumes and Cosmetics.
- The locations of enablers like drugstores and cosmetic stores and luxury stores.
Foursquare can help analysing location datas for Zurich, because using it not only provides
Coordinates of locations, it also helps to visualize the situation on a map.
The report and the tool helps to analyze and to understand the current situation 
for Pharmacies in the City of Zurich. But is is also a useful for future 
developments. Where should I open a Pharmacy? Where should I close a Pharmacy.
Are there Marketing instruments to make situation better? Should we - as a chain - 
cooperate with competitor or even buy them? Should we sell stores of focus on
other products we sell beside drugs? And so on. One important basis for decision
makers are facts, we can identify with the help of Foursquare.

### Audience

This report is useful for the marketing leaders in Pharmacy chains, for risk 
managers and for all investors of the Pharmacy business. For leaders and decission
makers.

### Description of the used data

Foursquare let me help finding places and informations wich are hard to find manually. But with python, some libraries and the great visualizing tool folio I can present the results to everybody. Also I use request for Foursquare Pharmacies and other locations/stores that have an influence on the Pharmacy revenue. I selected
- Pharmacy
- Perfume Shop 
- Medical Supply Store 
- Doctor s Office 
- Health Beauty Service
- Drugstore 
- Cosmetics Shop

I say that all these locations have an impact on the revenue of a Pharmcy in the city. It would be interesting to talk to experts and identify more
criteria with impact on city pharmacies.



In [1]:
import folium
import json
import requests
import pandas as pd
import math

In [2]:
#https://www.latlong.net/c/?lat=47.376888&long=8.541694
# Coordinates of Zurich:
#latitude  = 47.376888
#longitude =  8.541694

# Coordinates Pharmacy Victoria at Zurich:
latitude  = 47.374796 	
longitude =  8.538618

Show a map of Zurich:

In [3]:
locationmap = folium.Map(location=[latitude, longitude], zoom_start=15)
locationmap

In [4]:
CLIENT_ID = 'ZIVV2XBGULB4IKC2SFA33OT2WXMMAYEUNEMKP5NBZCSWL12U' # your Foursquare ID
CLIENT_SECRET = 'IAIN5OB2CYZ44STUMX0PDCKIMRTJ4OX3NUKEPA0O1U0XSY2M' # your Foursquare Secret
VERSION = '20190425' # Foursquare API version 20190425 20180605
#print('My Foursquare credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Foursquare access is defined.')

Foursquare access is defined.


Foursquare has maybe 200 sort of locations defined. I select here the important ones for the project and I give them colors:
    - Red is for locations with a negative impact on Pharmacies
    - Yellow is for Pharmacies
    - Green is for locations with a positive impact on Pharmacies    - 

In [5]:
data = [
    ['4bf58dd8d48988d10f951735', 'Pharmacy'             ,'yellow'], 
    ['52f2ab2ebcbc57f1066b8b23', 'Perfume Shop'         ,'red'   ], 
    ['58daa1558bbb0b01f18ec206', 'Medical Supply Store' ,'red'   ], 
    ['4bf58dd8d48988d177941735', 'Doctor s Office'      ,'green' ], 
    ['54541900498ea6ccd0202697', 'Health Beauty Service','green' ], 
    ['5745c2e4498e11e7bccabdbd', 'Drugstore'            ,'green' ], 
    ['4bf58dd8d48988d10c951735', 'Cosmetics Shop'       ,'green' ],  
] 
# Create the pandas DataFrame 
df_stores = pd.DataFrame(data, columns = ['categoryId', 'categoryname', 'dotcolor'])
df_stores 

,categoryId,categoryname,dotcolor
0,4bf58dd8d48988d10f951735,Pharmacy,yellow
1,52f2ab2ebcbc57f1066b8b23,Perfume Shop,red
2,58daa1558bbb0b01f18ec206,Medical Supply Store,red
3,4bf58dd8d48988d177941735,Doctor s Office,green
4,54541900498ea6ccd0202697,Health Beauty Service,green
5,5745c2e4498e11e7bccabdbd,Drugstore,green
6,4bf58dd8d48988d10c951735,Cosmetics Shop,green


I build a string with all the interesting Foursquare Id's:

In [6]:
categoryId = ''
for index, row in df_stores.iterrows():
    categoryId = categoryId  + row['categoryId'] + ','    
print (categoryId)

4bf58dd8d48988d10f951735,52f2ab2ebcbc57f1066b8b23,58daa1558bbb0b01f18ec206,4bf58dd8d48988d177941735,54541900498ea6ccd0202697,5745c2e4498e11e7bccabdbd,4bf58dd8d48988d10c951735,


I build the URL and use the string from above.

In [7]:
LIMIT      =  100 # limit of number of venues returned by Foursquare API
radius     = 1500 # define radius
categoryId = categoryId

url        = "https://api.foursquare.com/v2/venues/explore?"
url = url + '&client_id='    + CLIENT_ID     
url = url + '&client_secret='+ CLIENT_SECRET 
url = url + '&v='            + VERSION       
url = url + '&ll='           + str(latitude) +','+ str(longitude)
url = url + '&radius='       + str(radius)
url = url + '&limit='        + str(LIMIT)
url = url + '&categoryId='   + categoryId

url
# https://api.foursquare.com/v2/venues/VENUE_ID

'https://api.foursquare.com/v2/venues/explore?&client_id=ZIVV2XBGULB4IKC2SFA33OT2WXMMAYEUNEMKP5NBZCSWL12U&client_secret=IAIN5OB2CYZ44STUMX0PDCKIMRTJ4OX3NUKEPA0O1U0XSY2M&v=20190425&ll=47.374796,8.538618&radius=1500&limit=100&categoryId=4bf58dd8d48988d10f951735,52f2ab2ebcbc57f1066b8b23,58daa1558bbb0b01f18ec206,4bf58dd8d48988d177941735,54541900498ea6ccd0202697,5745c2e4498e11e7bccabdbd,4bf58dd8d48988d10c951735,'

Now I send the request to Foursquare. Foursquare sends a json string back.

In [8]:
results = requests.get(url).json()
results.keys() # dict_keys(['meta', 'response'])

dict_keys(['meta', 'response'])

I do not show you the json string.

In [9]:
# results

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
# nearby_venues.head()

In [12]:
# filter columns
filtered_columns = ['venue.name', 'venue.location.address','venue.location.city', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]



After some playing with datas I can show you the answer from Fourthsquare as a table:

In [13]:
nearby_venues.head(100)

,name,address,city,categories,lat,lng
0,Bahnhof Apotheke,Bahnhofplatz 15,Zürich,Pharmacy,47.377142,8.539919
1,Cartier,"Bahnhofstrasse 47,",Zürich,Jewelry Store,47.372098,8.538147
2,Wax in the City Zürich,Talstrasse 58,Zürich,Cosmetics Shop,47.370532,8.536119
3,Aesop,Oberdorfstr. 2,Zürich,Cosmetics Shop,47.368266,8.545824
4,The Barber Paradox,Rennweg 32,Zürich,Health & Beauty Service,47.373420,8.539794
5,Import Parfumerie,NaN,Zürich,Perfume Shop,47.375791,8.539074
6,Victoria Apotheke,Bahnhofstr. 71,Zürich,Pharmacy,47.374796,8.538618
7,Seretis Plastic Surgery,3 Lintheschergasse,Zürich,Doctor's Office,47.375122,8.537944
8,Yves Rocher Zürich,Bahnhofstrasse 87,Zürich,Health & Beauty Service,47.376316,8.539157
9,Kiehl's,Rennweg 23,Zürich,Cosmetics Shop,47.373075,8.539296


Do not read this. I just try to understand the JSON Data structure.

referralId                                            e-0-4b8ff66ef964a520926c33e3-0
reasons.count                                                                      0
reasons.items                      [{'summary': 'This spot is popular', 'type': '...
venue.id                                                    4b8ff66ef964a520926c33e3
venue.name                                                          Bahnhof Apotheke
venue.location.address                                               Bahnhofplatz 15
venue.location.lat                                                           47.3771
venue.location.lng                                                           8.53992
venue.location.labeledLatLngs      [{'label': 'display', 'lat': 47.37714158545192...
venue.location.distance                                                          278
venue.location.postalCode                                                       8001
venue.location.cc                                                                 CH
venue.location.city                                                           Zürich
venue.location.state                                                          Zürich
venue.location.country                                                       Schweiz
venue.location.formattedAddress              [Bahnhofplatz 15, 8001 Zürich, Schweiz]
venue.categories                   [{'id': '4bf58dd8d48988d10f951735', 'name': 'P...
venue.photos.count                                                                 0
venue.photos.groups                                                               []
venue.venuePage.id                                                               NaN
venue.location.crossStreet                                                       NaN
Name: 0, dtype: object
Bahnhof Apotheke [{'id': '4bf58dd8d48988d10f951735', 'name': 'Pharmacy', 'pluralName': 'Pharmacies', 'shortName': 'Pharmacy', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_', 'suffix': '.png'}, 'primary': True}]

I add the colors to the results.

In [14]:
List = [] 
for index, row in nearby_venues.iterrows():
    #print (row )
    #category = row["venue.categories"][0]['name']
    category  = row['categories']
    if "Pharmacy" in category:
        dotcolor ='yellow'
    elif 'Cosmetics Shop' in category:
        dotcolor ='red'
    elif 'Perfume Shop' in category:
        dotcolor ='red'    
    else:
        dotcolor ='green'
    print (dotcolor,'\t' ,category)
    List.append(dotcolor) 
    #print ('----------------------')


yellow 	 Pharmacy
green 	 Jewelry Store
red 	 Cosmetics Shop
red 	 Cosmetics Shop
green 	 Health & Beauty Service
red 	 Perfume Shop
yellow 	 Pharmacy
green 	 Doctor's Office
green 	 Health & Beauty Service
red 	 Cosmetics Shop
red 	 Cosmetics Shop
yellow 	 Pharmacy
green 	 Doctor's Office
green 	 Health & Beauty Service
red 	 Cosmetics Shop
green 	 Health & Beauty Service
yellow 	 Pharmacy
red 	 Cosmetics Shop
red 	 Cosmetics Shop
green 	 Doctor's Office
green 	 Health & Beauty Service
red 	 Cosmetics Shop
green 	 Health & Beauty Service
red 	 Cosmetics Shop
red 	 Cosmetics Shop
yellow 	 Pharmacy
yellow 	 Pharmacy
green 	 Health & Beauty Service
red 	 Cosmetics Shop
green 	 Health & Beauty Service
yellow 	 Pharmacy
green 	 Salon / Barbershop
green 	 Emergency Room
yellow 	 Pharmacy
green 	 Salon / Barbershop
green 	 Health & Beauty Service
red 	 Cosmetics Shop
green 	 Health & Beauty Service
yellow 	 Pharmacy
red 	 Cosmetics Shop
yellow 	 Pharmacy
green 	 Health & Beauty Service
green

Then I add the newly created List with colors (dotcolor) as Column to the Dataframe. 
Again:
- yellow for pharmacie
- green  for positive influence:
- red    for negative influence: Perfume Shop, Cosmetics Shop

In [15]:
# Add column
nearby_venues['dotcolor'] = List
nearby_venues

,name,address,city,categories,lat,lng,dotcolor
0,Bahnhof Apotheke,Bahnhofplatz 15,Zürich,Pharmacy,47.377142,8.539919,yellow
1,Cartier,"Bahnhofstrasse 47,",Zürich,Jewelry Store,47.372098,8.538147,green
2,Wax in the City Zürich,Talstrasse 58,Zürich,Cosmetics Shop,47.370532,8.536119,red
3,Aesop,Oberdorfstr. 2,Zürich,Cosmetics Shop,47.368266,8.545824,red
4,The Barber Paradox,Rennweg 32,Zürich,Health & Beauty Service,47.373420,8.539794,green
5,Import Parfumerie,NaN,Zürich,Perfume Shop,47.375791,8.539074,red
6,Victoria Apotheke,Bahnhofstr. 71,Zürich,Pharmacy,47.374796,8.538618,yellow
7,Seretis Plastic Surgery,3 Lintheschergasse,Zürich,Doctor's Office,47.375122,8.537944,green
8,Yves Rocher Zürich,Bahnhofstrasse 87,Zürich,Health & Beauty Service,47.376316,8.539157,green
9,Kiehl's,Rennweg 23,Zürich,Cosmetics Shop,47.373075,8.539296,red


In [16]:
# .drop(['B', 'C'], axis=1)
for col in nearby_venues.columns: 
    print(col) 
#nearby_venues.drop(columns=['venue.id', 'referralId', 'reasons.count'], axis=1)
# nearby_venues

name
address
city
categories
lat
lng
dotcolor


Here I create the Map of Zurich with the colored locations:

In [17]:
# for lat, lng, label in result['Latitude'], result['Longitude'], result['Neighbourhood']:
# label = folium.Popup(label, parse_html=True)
for ind in nearby_venues.index: 
    # print(result['Neighbourhood'][ind], result['Latitude'][ind],   result['Longitude'][ind])       
    #colorofcircle = nearby_venues['dotcolor'][ind]
              
    popupname = nearby_venues['categories'][ind] +' '+ nearby_venues['name'][ind]
    
    folium.CircleMarker(
        [nearby_venues['lat'][ind], nearby_venues['lng'][ind]],
        radius=5,
        popup=popupname,
        color=nearby_venues['dotcolor'][ind],
        fill=False,
        fill_color=nearby_venues['dotcolor'][ind],
        fill_opacity=0,
        parse_html=False).add_to(locationmap)  
locationmap

For each pharmacy (yellow dot), we calculate distances to other points. We us Newtons Law to calculate the force out of the distance.
Here, force can be negative, eg. for close competitors or positive for enables like doctor offices.
For each pharmacy we calculate the sum of forces. 

In [18]:
import math
list = []
for ind1 in nearby_venues.index: 
    lat1 = nearby_venues['lat'][ind1] *  90000 # approximately meters
    lng1 = nearby_venues['lng'][ind1] * 111000 # approximately meters
    #print (lat1,'\t', lng1)
    print (ind1)
    sum_force=0
    for ind2 in nearby_venues.index:
        lat2 = nearby_venues['lat'][ind2] *  90000 # approximately meters
        lng2 = nearby_venues['lng'][ind2] * 111000 # approximately meters
        #print ('\t', lat2, lng2)
        lat3 = abs(lat1-lat2) # *  90000 # approximately meters
        lng3 = abs(lat1-lat2) # * 111000 # approximately meters
        print (ind1, ind2, 'lat long diff\t', lat3,'\t', lng3)
        distance = math.sqrt(math.pow(lat3, 2) + math.pow(lng3, 2))  # Pythagoras
        
        if distance==0:
            force = 0
        else:
            force = 1/(math.pow(distance,2) ) * 1000                # Newtons Law 
  
        if   nearby_venues['dotcolor'][ind2] == 'red':
            sign = -1
        elif nearby_venues['dotcolor'][ind2] == 'yellow':            # Other pharmacies are competitors
            sign = -1
        else:
            sign = +1
            
        force     = force     * sign         
        sum_force = sum_force + force
        print (ind1, ind2, 'lat long diff\t', lat1, lat2, lat3,'\t', lng1, lng2, lng3)
        print (ind1, ind2, "%8.2f m\t %+8.8f  \t sum force:%+8.8f" % (distance, force, sum_force))
        
    if   nearby_venues['dotcolor'][ind1] == 'yellow':
        sum_force = sum_force + force
    else:
        sum_force = 0                                   # we only calculate pharmacies
        
    print(ind1, "----------------%8.2f m\t %+8.8f  \t sum force:%+8.8f" % (distance, force, sum_force))
    

    list.append(sum_force)
nearby_venues['force'] = list   
        
        

0
0 0 lat long diff	 0.0 	 0.0
0 0 lat long diff	 4263942.742690673 4263942.742690673 0.0 	 947930.9918890799 947930.9918890799 0.0
0 0     0.00 m	 +0.00000000  	 sum force:+0.00000000
0 1 lat long diff	 453.9036522405222 	 453.9036522405222
0 1 lat long diff	 4263942.742690673 4263488.839038433 453.9036522405222 	 947930.9918890799 947734.3483801711 453.9036522405222
0 1   641.92 m	 +0.00242685  	 sum force:+0.00242685
0 2 lat long diff	 594.821893453598 	 594.821893453598
0 2 lat long diff	 4263942.742690673 4263347.9207972195 594.821893453598 	 947930.9918890799 947509.1860991735 594.821893453598
0 2   841.21 m	 -0.00141318  	 sum force:+0.00101367
0 3 lat long diff	 798.815507591702 	 798.815507591702
0 3 lat long diff	 4263942.742690673 4263143.927183081 798.815507591702 	 947930.9918890799 948586.4471395698 798.815507591702
0 3  1129.70 m	 -0.00078357  	 sum force:+0.00023010
0 4 lat long diff	 334.94269067328423 	 334.94269067328423
0 4 lat long diff	 4263942.742690673 4263607.8

1 37 lat long diff	 4263488.839038433 4264393.544999999 904.7059615664184 	 947734.3483801711 948824.4924 904.7059615664184
1 37  1279.45 m	 +0.00061088  	 sum force:+3.96627912
1 38 lat long diff	 1058.9832731038332 	 1058.9832731038332
1 38 lat long diff	 4263488.839038433 4264547.822311536 1058.9832731038332 	 947734.3483801711 947114.93240361 1058.9832731038332
1 38  1497.63 m	 -0.00044585  	 sum force:+3.96583327
1 39 lat long diff	 990.7197448099032 	 990.7197448099032
1 39 lat long diff	 4263488.839038433 4264479.5587832425 990.7197448099032 	 947734.3483801711 946739.6455347339 990.7197448099032
1 39  1401.09 m	 -0.00050941  	 sum force:+3.96532386
1 40 lat long diff	 538.3682474661618 	 538.3682474661618
1 40 lat long diff	 4263488.839038433 4262950.470790966 538.3682474661618 	 947734.3483801711 948836.7006761208 538.3682474661618
1 40   761.37 m	 -0.00172509  	 sum force:+3.96359877
1 41 lat long diff	 545.1580384327099 	 545.1580384327099
1 41 lat long diff	 4263488.8390384

3 12 lat long diff	 617.1997653087601 	 617.1997653087601
3 12 lat long diff	 4263143.927183081 4263761.12694839 617.1997653087601 	 948586.4471395698 947714.9127024006 617.1997653087601
3 12   872.85 m	 +0.00131256  	 sum force:-0.00993127
3 13 lat long diff	 483.02481691911817 	 483.02481691911817
3 13 lat long diff	 4263143.927183081 4263626.9520000005 483.02481691911817 	 948586.4471395698 947561.3457 483.02481691911817
3 13   683.10 m	 +0.00214304  	 sum force:-0.00778823
3 14 lat long diff	 649.2586257793009 	 649.2586257793009
3 14 lat long diff	 4263143.927183081 4263793.185808861 649.2586257793009 	 948586.4471395698 947780.342534458 649.2586257793009
3 14   918.19 m	 -0.00118614  	 sum force:-0.00897436
3 15 lat long diff	 595.4528169184923 	 595.4528169184923
3 15 lat long diff	 4263143.927183081 4263739.38 595.4528169184923 	 948586.4471395698 947744.8620000001 595.4528169184923
3 15   842.10 m	 +0.00141018  	 sum force:-0.00756418
3 16 lat long diff	 644.1034978637472 	 64

4 49 lat long diff	 630.7346130898222 	 630.7346130898222
4 49 lat long diff	 4263607.8 4262977.06538691 630.7346130898222 	 947917.1340000001 949447.9916107326 630.7346130898222
4 49   891.99 m	 +0.00125683  	 sum force:-7.20437808
4 50 lat long diff	 268.03800000064075 	 268.03800000064075
4 50 lat long diff	 4263607.8 4263339.761999999 268.03800000064075 	 947917.1340000001 945949.104 268.03800000064075
4 50   379.06 m	 +0.00695949  	 sum force:-7.19741860
4 51 lat long diff	 349.909652444534 	 349.909652444534
4 51 lat long diff	 4263607.8 4263957.709652444 349.909652444534 	 947917.1340000001 947892.7355005322 349.909652444534
4 51   494.85 m	 +0.00408374  	 sum force:-7.19333485
4 ----------------  494.85 m	 +0.00408374  	 sum force:+0.00000000
5
5 0 lat long diff	 121.55546574946493 	 121.55546574946493
5 0 lat long diff	 4263821.187224924 4263942.742690673 121.55546574946493 	 947837.2161289675 947930.9918890799 121.55546574946493
5 0   171.91 m	 -0.03383927  	 sum force:-0.033

6 23 lat long diff	 33.138068448752165 	 33.138068448752165
6 23 lat long diff	 4263731.598523754 4263764.736592202 33.138068448752165 	 947786.5741360105 948322.9175308252 33.138068448752165
6 23    46.86 m	 -0.45531884  	 sum force:+8.59870708
6 24 lat long diff	 311.76675909012556 	 311.76675909012556
6 24 lat long diff	 4263731.598523754 4264043.365282844 311.76675909012556 	 947786.5741360105 947792.2628176509 311.76675909012556
6 24   440.90 m	 -0.00514411  	 sum force:+8.59356296
6 25 lat long diff	 131.98852375429124 	 131.98852375429124
6 25 lat long diff	 4263731.598523754 4263599.609999999 131.98852375429124 	 947786.5741360105 948368.1269999999 131.98852375429124
6 25   186.66 m	 -0.02870104  	 sum force:+8.56486192
6 26 lat long diff	 202.5284989187494 	 202.5284989187494
6 26 lat long diff	 4263731.598523754 4263934.127022672 202.5284989187494 	 947786.5741360105 948360.6872889838 202.5284989187494
6 26   286.42 m	 -0.01218983  	 sum force:+8.55267209
6 27 lat long diff	 

...

44 4 lat long diff	 180.34507447946817 	 180.34507447946817
44 4 lat long diff	 4263427.45492552 4263607.8 180.34507447946817 	 946023.987526286 947917.1340000001 180.34507447946817
44 4   255.05 m	 +0.01537310  	 sum force:+0.06092332
44 5 lat long diff	 393.73229940328747 	 393.73229940328747
44 5 lat long diff	 4263427.45492552 4263821.187224924 393.73229940328747 	 946023.987526286 947837.2161289675 393.73229940328747
44 5   556.82 m	 -0.00322528  	 sum force:+0.05769804
44 6 lat long diff	 304.1435982333496 	 304.1435982333496
44 6 lat long diff	 4263427.45492552 4263731.598523754 304.1435982333496 	 946023.987526286 947786.5741360105 304.1435982333496
44 6   430.12 m	 -0.00540521  	 sum force:+0.05229283
44 7 lat long diff	 333.53407447971404 	 333.53407447971404
44 7 lat long diff	 4263427.45492552 4263760.989 333.53407447971404 	 946023.987526286 947711.739600007 333.53407447971404
44 7   471.69 m	 +0.00449458  	 sum force:+0.05678741
44 8 lat long diff	 440.96707447990775 	 44

45 35 lat long diff	 4262812.481643132 4263036.3 223.81835686787963 	 946962.6367092133 947148.5700000001 223.81835686787963
45 35   316.53 m	 +0.00998110  	 sum force:+0.00734886
45 36 lat long diff	 244.57577406894416 	 244.57577406894416
45 36 lat long diff	 4262812.481643132 4263057.057417201 244.57577406894416 	 946962.6367092133 948500.2716235856 244.57577406894416
45 36   345.88 m	 -0.00835878  	 sum force:-0.00100992
45 37 lat long diff	 1581.06335686706 	 1581.06335686706
45 37 lat long diff	 4262812.481643132 4264393.544999999 1581.06335686706 	 946962.6367092133 948824.4924 1581.06335686706
45 37  2235.96 m	 +0.00020002  	 sum force:-0.00080990
45 38 lat long diff	 1735.3406684044749 	 1735.3406684044749
45 38 lat long diff	 4262812.481643132 4264547.822311536 1735.3406684044749 	 946962.6367092133 947114.93240361 1735.3406684044749
45 38  2454.14 m	 -0.00016604  	 sum force:-0.00097594
45 39 lat long diff	 1667.0771401105449 	 1667.0771401105449
45 39 lat long diff	 4262812

47 26   926.87 m	 -0.00116402  	 sum force:-0.11408178
47 27 lat long diff	 762.8702817158774 	 762.8702817158774
47 27 lat long diff	 4263278.729718284 4264041.6 762.8702817158774 	 949694.6995093122 948392.8689 762.8702817158774
47 27  1078.86 m	 +0.00085915  	 sum force:-0.11322263
47 28 lat long diff	 767.9524272279814 	 767.9524272279814
47 28 lat long diff	 4263278.729718284 4264046.682145512 767.9524272279814 	 949694.6995093122 947104.9379723478 767.9524272279814
47 28  1086.05 m	 -0.00084782  	 sum force:-0.11407044
47 29 lat long diff	 69.7377182841301 	 69.7377182841301
47 29 lat long diff	 4263278.729718284 4263208.992 69.7377182841301 	 949694.6995093122 947375.9645999966 69.7377182841301
47 29    98.62 m	 +0.10280981  	 sum force:-0.01126064
47 30 lat long diff	 315.6191067751497 	 315.6191067751497
47 30 lat long diff	 4263278.729718284 4263594.348825059 315.6191067751497 	 949694.6995093122 946738.5109595032 315.6191067751497
47 30   446.35 m	 -0.00501930  	 sum force:-

49 33 lat long diff	 4262977.06538691 4263096.580005711 119.51461880095303 	 949447.9916107326 948528.8379939246 119.51461880095303
49 33   169.02 m	 -0.03500483  	 sum force:-0.04257587
49 34 lat long diff	 148.39414445869625 	 148.39414445869625
49 34 lat long diff	 4262977.06538691 4263125.459531369 148.39414445869625 	 949447.9916107326 947492.8862464691 148.39414445869625
49 34   209.86 m	 +0.02270578  	 sum force:-0.01987009
49 35 lat long diff	 59.23461308982223 	 59.23461308982223
49 35 lat long diff	 4262977.06538691 4263036.3 59.23461308982223 	 949447.9916107326 947148.5700000001 59.23461308982223
49 35    83.77 m	 +0.14250132  	 sum force:+0.12263123
49 36 lat long diff	 79.99203029088676 	 79.99203029088676
49 36 lat long diff	 4262977.06538691 4263057.057417201 79.99203029088676 	 949447.9916107326 948500.2716235856 79.99203029088676
49 36   113.13 m	 -0.07814057  	 sum force:+0.04449066
49 37 lat long diff	 1416.4796130890027 	 1416.4796130890027
49 37 lat long diff	 426

51 24 lat long diff	 85.65563039947301 	 85.65563039947301
51 24 lat long diff	 4263957.709652444 4264043.365282844 85.65563039947301 	 947892.7355005322 947792.2628176509 85.65563039947301
51 24   121.14 m	 -0.06814879  	 sum force:-2.88776285
51 25 lat long diff	 358.0996524449438 	 358.0996524449438
51 25 lat long diff	 4263957.709652444 4263599.609999999 358.0996524449438 	 947892.7355005322 948368.1269999999 358.0996524449438
51 25   506.43 m	 -0.00389908  	 sum force:-2.89166193
51 26 lat long diff	 23.582629771903157 	 23.582629771903157
51 26 lat long diff	 4263957.709652444 4263934.127022672 23.582629771903157 	 947892.7355005322 948360.6872889838 23.582629771903157
51 26    33.35 m	 -0.89905350  	 sum force:-3.79071544
51 27 lat long diff	 83.89034755527973 	 83.89034755527973
51 27 lat long diff	 4263957.709652444 4264041.6 83.89034755527973 	 947892.7355005322 948392.8689 83.89034755527973
51 27   118.64 m	 +0.07104704  	 sum force:-3.71966839
51 28 lat long diff	 88.972493

Now we have an enhanced table with a force column.

In [19]:
nearby_venues

,name,address,city,categories,lat,lng,dotcolor,force
0,Bahnhof Apotheke,Bahnhofplatz 15,Zürich,Pharmacy,47.377142,8.539919,yellow,-2.759539
1,Cartier,"Bahnhofstrasse 47,",Zürich,Jewelry Store,47.372098,8.538147,green,0.000000
2,Wax in the City Zürich,Talstrasse 58,Zürich,Cosmetics Shop,47.370532,8.536119,red,0.000000
3,Aesop,Oberdorfstr. 2,Zürich,Cosmetics Shop,47.368266,8.545824,red,0.000000
4,The Barber Paradox,Rennweg 32,Zürich,Health & Beauty Service,47.373420,8.539794,green,0.000000
5,Import Parfumerie,NaN,Zürich,Perfume Shop,47.375791,8.539074,red,0.000000
6,Victoria Apotheke,Bahnhofstr. 71,Zürich,Pharmacy,47.374796,8.538618,yellow,8.584538
7,Seretis Plastic Surgery,3 Lintheschergasse,Zürich,Doctor's Office,47.375122,8.537944,green,0.000000
8,Yves Rocher Zürich,Bahnhofstrasse 87,Zürich,Health & Beauty Service,47.376316,8.539157,green,0.000000
9,Kiehl's,Rennweg 23,Zürich,Cosmetics Shop,47.373075,8.539296,red,0.000000


We are only intersted in the location value of pharmacies. We sort the pharmacies so that the pharmacies with the
best force are at the top.

In [20]:
nearby_venues.query('dotcolor == "yellow"', inplace = True) 
nearby_venues = nearby_venues.sort_values(by ='force', ascending=False)
nearby_venues

,name,address,city,categories,lat,lng,dotcolor,force
30,Berg-Apotheke,NaN,Zürich,Pharmacy,47.373271,8.529176,yellow,50.270875
40,Dr. Andres Apotheke Stadelhofen,Goethestrasse 22,Zürich,Pharmacy,47.366116,8.548078,yellow,14.351097
6,Victoria Apotheke,Bahnhofstr. 71,Zürich,Pharmacy,47.374796,8.538618,yellow,8.584538
47,DROPA Apotheke & Post Hottingen,"Freiestrasse 55, Beim Hottingerplatz",Zürich,Pharmacy,47.369764,8.555808,yellow,0.488604
33,Odeon Apotheke,"Limmatquai 2, Bellevueplatz",Zürich,Pharmacy,47.367740,8.545305,yellow,0.301823
45,Topwell Apotheke-Drogerie,Tessinerplatz 10,Zürich,Pharmacy,47.364583,8.531195,yellow,0.114963
38,TopPharm Limmatplatz Apotheke,Limmatstrasse 119,Zürich,Pharmacy,47.383865,8.532567,yellow,-0.085625
25,Rosen Apotheke,Niederdorfstrasse 11,Zürich,Pharmacy,47.373329,8.543857,yellow,-2.695804
0,Bahnhof Apotheke,Bahnhofplatz 15,Zürich,Pharmacy,47.377142,8.539919,yellow,-2.759539
11,Coop Vitality Zürich Bahnhofstrasse,Bahnhofstrasse 81,Zürich,Pharmacy,47.375954,8.539052,yellow,-3.727254


Known Problem:
    If the difference between two latitudes or two longitudes is very small, the error in the distance calculation is big. 
    Distances are calculated much smaller than in reality. This leads to wrong results in rare cases. 

Thank you for reading all the code.
Pascal 3.8.2020